# Predicting Enron Spam Emails using Supervised Learning

## DS-GA 1001: Introduction to Data Science Final Project

### Scripts

## Supervised Learning Models

Created On: 11/30/2020

Modified On: 12/01/2020

### Description

This script establishes various supervised learning models for the `emails_cleaned.csv` dataset.